In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# Preprocess the text data
def preprocess_text(text):
    # Lowercasing the text
    text = text.lower()
    # Remove non-alphanumeric characters (keeping spaces)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove digits
    text = re.sub(r'\d+', '', text)
    return text

# File paths for train and test data
train_file_path = '/content/Tam-SA-train.csv'
test_file_path = '/content/Tam-SA-test-without-labels.csv'

# Reading the data
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Check the column names in the train dataset
print("Train Data Columns:", train_data.columns)

# Adjusting column name based on your dataset
# The 'Label' column contains the sentiment labels, so we'll use that
label_encoder = LabelEncoder()
train_data['labels_encoded'] = label_encoder.fit_transform(train_data['Label'])

# Preprocess the text in both train and test datasets
train_data['Text'] = train_data['Text'].apply(preprocess_text)
test_data['Text'] = test_data['Text'].apply(preprocess_text)

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X = tfidf_vectorizer.fit_transform(train_data['Text']).toarray()
y = train_data['labels_encoded']

# Splitting the dataset into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Handling class imbalance using SMOTE (Synthetic Minority Oversampling Technique)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_res, y_res)

# Best model from grid search
clf_best = grid_search.best_estimator_

# Train the model using the best hyperparameters
clf_best.fit(X_res, y_res)

# Evaluate the model
y_pred = clf_best.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Classification report
report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)
print('Classification Report:')
print(report)

# Save the trained model, TF-IDF vectorizer, and label encoder
joblib.dump(clf_best, 'random_forest_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Function to test the model with new text input
def test_model(text, clf, tfidf_vectorizer, label_encoder):
    text_tfidf = tfidf_vectorizer.transform([preprocess_text(text)]).toarray()
    prediction = clf.predict(text_tfidf)
    predicted_label = label_encoder.inverse_transform(prediction)
    return predicted_label[0]

# Apply the model to the test dataset
test_data['Predicted'] = test_data['Text'].apply(lambda text: test_model(text, clf_best, tfidf_vectorizer, label_encoder))

# Save the predictions to a new CSV file
output_file_path = '/content/drive/MyDrive/codalab/tam_prediction.csv'
test_data.to_csv(output_file_path, index=False)

print(f'Predictions saved to {output_file_path}')


Train Data Columns: Index(['Text', 'Label'], dtype='object')
Validation Accuracy: 0.5697991967871486
Classification Report:
                precision    recall  f1-score   support

Mixed_feelings       0.25      0.19      0.22       694
      Negative       0.29      0.42      0.34       776
      Positive       0.75      0.73      0.74      3695
 unknown_state       0.40      0.38      0.39      1060

      accuracy                           0.57      6225
     macro avg       0.42      0.43      0.42      6225
  weighted avg       0.58      0.57      0.57      6225

Predictions saved to /content/drive/MyDrive/codalab/tam_prediction.csv
